In [1]:
import pandas as pd
import pefile
from capstone import *

In [2]:
pe = pefile.PE("./MALWR/sample_vg655_25th.exe")
print('SECCIONES \t')
for section in pe.sections:
    print(section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize), section.SizeOfRawData)

SECCIONES 	
b'.text\x00\x00\x00' 0x1000 0x69b0 28672
b'.rdata\x00\x00' 0x8000 0x5f70 24576
b'.data\x00\x00\x00' 0xe000 0x1958 8192
b'.rsrc\x00\x00\x00' 0x10000 0x349fa0 3448832


In [3]:
for entry in pe.DIRECTORY_ENTRY_IMPORT:
    print('LLAMADAS A DLL \t')
    print (entry.dll)
    print('LLAMADAS A FUNCIONES \t')
    for function in entry.imports:
        print ('\t', function.name)

LLAMADAS A DLL 	
b'KERNEL32.dll'
LLAMADAS A FUNCIONES 	
	 b'GetFileAttributesW'
	 b'GetFileSizeEx'
	 b'CreateFileA'
	 b'InitializeCriticalSection'
	 b'DeleteCriticalSection'
	 b'ReadFile'
	 b'GetFileSize'
	 b'WriteFile'
	 b'LeaveCriticalSection'
	 b'EnterCriticalSection'
	 b'SetFileAttributesW'
	 b'SetCurrentDirectoryW'
	 b'CreateDirectoryW'
	 b'GetTempPathW'
	 b'GetWindowsDirectoryW'
	 b'GetFileAttributesA'
	 b'SizeofResource'
	 b'LockResource'
	 b'LoadResource'
	 b'MultiByteToWideChar'
	 b'Sleep'
	 b'OpenMutexA'
	 b'GetFullPathNameA'
	 b'CopyFileA'
	 b'GetModuleFileNameA'
	 b'VirtualAlloc'
	 b'VirtualFree'
	 b'FreeLibrary'
	 b'HeapAlloc'
	 b'GetProcessHeap'
	 b'GetModuleHandleA'
	 b'SetLastError'
	 b'VirtualProtect'
	 b'IsBadReadPtr'
	 b'HeapFree'
	 b'SystemTimeToFileTime'
	 b'LocalFileTimeToFileTime'
	 b'CreateDirectoryA'
	 b'GetStartupInfoA'
	 b'SetFilePointer'
	 b'SetFileTime'
	 b'GetComputerNameW'
	 b'GetCurrentDirectoryA'
	 b'SetCurrentDirectoryA'
	 b'GlobalAlloc'
	 b'LoadLibrar

In [4]:
print("TimeDateStamp : " + pe.FILE_HEADER.dump_dict()['TimeDateStamp']['Value'].split('[')[1][:-1])
print("TimeDateStamp: "+hex(pe.FILE_HEADER.TimeDateStamp))

TimeDateStamp : Sat Nov 20 09:05:05 2010 UTC
TimeDateStamp: 0x4ce78f41


In [5]:
pe2 = pefile.PE("./MALWR/sample_qwrty_dk2")
print('SECCIONES \t')
for section in pe2.sections:
    print(section.Name, hex(section.VirtualAddress), hex(section.Misc_VirtualSize), section.SizeOfRawData)

SECCIONES 	
b'UPX0\x00\x00\x00\x00' 0x1000 0x5000 0
b'UPX1\x00\x00\x00\x00' 0x6000 0x1000 4096
b'.rsrc\x00\x00\x00' 0x7000 0x1000 512


In [6]:
for entry in pe2.DIRECTORY_ENTRY_IMPORT:
    print('LLAMADAS A DLL \t')
    print (entry.dll)
    print('LLAMADAS A FUNCIONES \t')
    for function in entry.imports:
        print ('\t', function.name)

LLAMADAS A DLL 	
b'KERNEL32.DLL'
LLAMADAS A FUNCIONES 	
	 b'LoadLibraryA'
	 b'ExitProcess'
	 b'GetProcAddress'
	 b'VirtualProtect'
LLAMADAS A DLL 	
b'MSVCRT.dll'
LLAMADAS A FUNCIONES 	
	 b'atol'
LLAMADAS A DLL 	
b'SHELL32.dll'
LLAMADAS A FUNCIONES 	
	 b'SHChangeNotify'
LLAMADAS A DLL 	
b'USER32.dll'
LLAMADAS A FUNCIONES 	
	 b'LoadStringA'
LLAMADAS A DLL 	
b'WS2_32.dll'
LLAMADAS A FUNCIONES 	
	 b'closesocket'


In [7]:
print("TimeDateStamp : " + pe2.FILE_HEADER.dump_dict()['TimeDateStamp']['Value'].split('[')[1][:-1])
print("TimeDateStamp: "+hex(pe2.FILE_HEADER.TimeDateStamp))

TimeDateStamp : Thu May 14 17:12:40 2009 UTC
TimeDateStamp: 0x4a0c5108


1. Las llamadas a las dll son una mas en sample_qwrty_dk2 pero esto no quiere decir que sea sospechoso ya que el otro archivo utiliza las mismas librerias exceptuando una, lo realmente curioso es la cantidad de funciones que sample_vg655_25th.exe utiliza, son significativamente mayores sin mencionar el uso de funciones poco comunes dentro de la dll de msvcrt.dll.

2. El uso de Upx implica el uso de un software con el mismo nombre de upx que es muy comun en la creacion de malwares, lo que indicaria que puede estar relacionado a malware.

3. Estan relacionadas a las categorias de lee y escribir archivos asi como la obtencion de informacion


In [12]:
apis = pd.DataFrame(data={'APIs':['ReadFile','CreateFile','CreateFileA','GetShortPathNameA','GetFileAttributesW'],'Categorias':['Read/Write Files','Read/Write Files','Read/Write Files','Get File Information','Get File Information']})
apis

,APIs,Categorias
0,ReadFile,Read/Write Files
1,CreateFile,Read/Write Files
2,CreateFileA,Read/Write Files
3,GetShortPathNameA,Get File Information
4,GetFileAttributesW,Get File Information


# 4.
![imagen1](s1.jpg)

5. Es para el registro y administracion de servicios de windows, en el caso del ejemplo se puede notar el uso para crear, registrar e iniciar un serivicio lo cual es sospechoso.

6. Es un servicio para funcoines criptograficas y el uso de un contenedor de claves usado por el usuario.

7. Si es sospechoso por la cantidad de apis que utiliza, las cuales son relacionadas a la creacion, movimiento y escritura de archivos lo cual involucra ejecutar operaciones peligorsas dentro del sistema, sin mencionar el uso de servicios criptograficos dentro de la dll de registro y administracion de servicios del sistema. El proposito de este posible malware se basa en los arvhicos del usuario para hacer uso de estos.

8. Si coincide el hash y el malware que se detecta es omo_im_not_ramsomware_xd el cual nos da la clara idea de que es un ramsomware el cual bloquea la compu para el acceso del usuario encriptando sus archivos preciados y exigiendo un pago por la liberacion de los mismos.
![imagen2](s2.jpg)

9. Exactamente el ramsomware hace uso de los archivos del usuario para realizar acciones que perjudican al usuario que fue justamente lo que se sospecho en el inciso 7 que era el uso de los archivos del usuario.
![s3](d.png)
![s4](s3.jpeg)